In [1]:
######## snakemake preamble start (automatically inserted, do not edit) ########
library(methods)
Snakemake <- setClass(
    "Snakemake",
    slots = c(
        input = "list",
        output = "list",
        params = "list",
        wildcards = "list",
        threads = "numeric",
        log = "list",
        resources = "list",
        config = "list",
        rule = "character",
        bench_iteration = "numeric",
        scriptdir = "character",
        source = "function"
    )
)
snakemake <- Snakemake(
    input = list('rds' = 'out/ipynb/compute_differential_methylation/all_unpaired.rds'),
    output = list('xlsx' = 'out/ipynb/visualize_heatmap_by_gene/CUX2.xlsx'),
    params = list(),
    wildcards = list('dev_unpaired', "design" = 'dev_unpaired'),
    threads = 2,
    # log = list('out/ipynb/compute_differential_methylation/dev_unpaired.rds', 'out/ipynb/compute_differential_methylation/dev_unpaired.r.ipynb', "rds" = 'out/ipynb/compute_differential_methylation/dev_unpaired.rds', "notebook" = 'out/ipynb/compute_differential_methylation/dev_unpaired.r.ipynb'),
    resources = list('tmpdir', "tmpdir" = '/tmp'),
    config = list(),
    rule = 'ipynb_compute_differential_methylation',
    bench_iteration = as.numeric(NA),
    scriptdir = '/mnt/thymus/synoSalva/illumina_sequencing_data/mw/mw-oncodiag/src/snakemake/rules/../../ipynb',
    source = function(...){
        wd <- getwd()
        setwd(snakemake@scriptdir)
        source(...)
        setwd(wd)
    }
)
setwd('/mnt/thymus/synoSalva/illumina_sequencing_data/mw/mw-oncodiag/');

######## snakemake preamble end #########


In [2]:
# start coding here

In [3]:
IRdisplay::display_markdown(
    sprintf(
"_Visualize heatmap by gene_

__Oncodiag project__

[Guillaume Charbonnier](mailto:gc.bioinfo@gmail.com)

%s",
        format(Sys.Date(), "%Y-%m-%d")
    )
)

_Visualize heatmap by gene_

__Oncodiag project__

[Guillaume Charbonnier](mailto:gc.bioinfo@gmail.com)

2023-10-15

# Abstract

The aim of this analysis is to produce heatmaps for all differentially methylated CG for the key genes of interest: SLC23A2, ANXA3 and CUX2.

# Methods

## Load dependencies

### Packages

In [4]:
if (!require("BiocManager", quietly = TRUE)) {
    install.packages(
        "BiocManager",
        quiet = TRUE
    )
}
packages <- c(
  # "data.table",
  "dplyr",
  "openxlsx",
  # "DT",
  # "methylSig",
  "DSS",
  "bsseq",
  # "ChIPpeakAnno",
  "ChIPseeker",
  # "methyAnalysis",
  "org.Hs.eg.db",
  "TxDb.Hsapiens.UCSC.hg19.knownGene",
  "EnsDb.Hsapiens.v75", #test
  # "EnhancedVolcano",
  # "data.table",
  "ggpubr",
  "ComplexHeatmap",
  "tidyr"
)
BiocManager::install(
    packages,
    update = FALSE,
    quiet = TRUE,
    Ncpus = parallel::detectCores()
)
invisible(
    lapply(
      packages,
      library,
      character.only = TRUE
    )
)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org



Bioconductor version 3.17 (BiocManager 1.30.22), R 4.3.1 (2023-06-16)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'dplyr' 'openxlsx' 'DSS' 'bsseq' 'ChIPseeker'
  'org.Hs.eg.db' 'TxDb.Hsapiens.UCSC.hg19.knownGene' 'ggpubr' 'ComplexHeatmap'
  'tidyr'”
Installing package(s) 'EnsDb.Hsapiens.v75'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘pack

In [5]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.2 LTS

Matrix products: default
BLAS/LAPACK: /cobelix/spicuglia/mambaforge/envs/rkernel_diff_meth_v2/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
 [1] grid      parallel  stats4    stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] tidyr_1.3.0                            
 [2] ComplexHeatmap_2.16.0                  
 [3] ggpubr_0.6.0                           
 [4] ggplot2_3.4.4                          
 [5] EnsDb.Hsapiens.v75_2.99.0              
 [6] en

In [6]:
set_plot_dim <- function(
    width = 16,
    height = 9
) {
    options(
        #jupyter.plot_scale=1,
        repr.plot.width = width,
        repr.plot.height = height
    )
}
set_plot_dim()

### Data

In [7]:
# dml_res <- readRDS(snakemake@input$rds)
dml_res <- readRDS("out/ipynb/compute_differential_methylation/all_unpaired.rds")
dim(dml_res)

[1] 55329507       11

In [8]:
dml_signif_res = callDML(
    dml_res,
    delta = 0.2,
    p.threshold = 0.001
)
dim(dml_signif_res)

[1] 1144667      12

In [9]:
# We need to move to the assay directory, else subsetting will not work
setwd("out/ipynb/compute_differential_methylation/all_unpairedhdf5a/")

In [10]:
se <- readRDS("se.rds")
se

An object of type 'BSseq' with
  55628636 methylation loci
  30 samples
has not been smoothed
Some assays are HDF5Array-backed

In [11]:
# Temp cell to investigate missing NBEAL2 downstream
# We check that we the probes are differential -> OK
dml_signif_res[dml_signif_res$chr %in% 3 & grepl("^470510", dml_signif_res$pos),]

,chr,pos,mu1,mu2,diff,diff.se,stat,phi1,phi2,pval,fdr,postprob.overThreshold
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9560535,3,47051012,0.5743813,0.06636666,0.5080146,0.03707762,13.70138,0.1252186,0.02358673,9.961120e-43,1.846811e-39,1
9560537,3,47051033,0.5755928,0.06700273,0.5085901,0.03673537,13.84470,0.1882280,0.02299715,1.369571e-43,2.799944e-40,1
9560538,3,47051034,0.5750143,0.06717295,0.5078413,0.03675559,13.81671,0.1300390,0.03430088,2.020957e-43,4.057424e-40,1
9560539,3,47051039,0.5750143,0.06717295,0.5078413,0.03669627,13.83904,0.1328551,0.02199048,1.481639e-43,3.019015e-40,1
9560540,3,47051040,0.5750143,0.06717295,0.5078413,0.03669650,13.83896,0.1484823,0.03756926,1.483451e-43,3.022596e-40,1
9560541,3,47051043,0.5750143,0.06717295,0.5078413,0.03668034,13.84506,0.1405560,0.02297326,1.362753e-43,2.786788e-40,1
9560542,3,47051044,0.5750143,0.06717295,0.5078413,0.03677695,13.80869,0.1199720,0.03151183,2.259141e-43,4.508626e-40,1
9560543,3,47051072,0.5739843,0.06764876,0.5063355,0.03645183,13.89054,0.1284906,0.03834277,7.228958e-44,1.526446e-40,1
9560544,3,47051073,0.5737090,0.06770263,0.5060064,0.03661123,13.82107,0.1278575,0.03457954,1.902218e-43,3.832105e-40,1


## Subset only significant probes

In [12]:
signif_se <- subsetByOverlaps(
    se,
    GRanges(
        seqnames = dml_signif_res$chr,
        IRanges(
            start = dml_signif_res$pos,
            width = 1
        )
    )
)
signif_se

An object of type 'BSseq' with
  1144667 methylation loci
  30 samples
has not been smoothed
Some assays are HDF5Array-backed

In [13]:
rowRanges(signif_se)

GRanges object with 1144667 ranges and 0 metadata columns:
            seqnames    ranges strand
               <Rle> <IRanges>  <Rle>
        [1]        1     59275      *
        [2]        1     85528      *
        [3]        1    526252      *
        [4]        1    526253      *
        [5]        1    526270      *
        ...      ...       ...    ...
  [1144663]        Y  58824332      *
  [1144664]        Y  58828517      *
  [1144665]        Y  58852674      *
  [1144666]        Y  58878151      *
  [1144667]        Y  58878233      *
  -------
  seqinfo: 25 sequences from an unspecified genome; no seqlengths

In [14]:
dml_signif_res[order(dml_signif_res$chr, dml_signif_res$pos),]

,chr,pos,mu1,mu2,diff,diff.se,stat,phi1,phi2,pval,fdr,postprob.overThreshold
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
910,1,59275,0.4445518,0.7888945,-0.3443427,0.01464996,-23.504681,0.05147487,0.03319496,3.653067e-122,1.962353e-116,1.0000000
1165,1,85528,0.2659917,0.6063652,-0.3403735,0.04387451,-7.757886,0.05192969,0.03470666,8.635638e-15,3.611938e-13,0.9993115
2590,1,526252,0.5202774,0.8422268,-0.3219494,0.03771890,-8.535494,0.05212250,0.03286102,1.395583e-17,9.272045e-16,0.9993878
2591,1,526253,0.5202774,0.8422268,-0.3219494,0.03887717,-8.281195,0.06097534,0.03327580,1.219453e-16,6.942488e-15,0.9991460
2592,1,526270,0.4837674,0.8201285,-0.3363611,0.03571735,-9.417302,0.04958166,0.03435590,4.628257e-21,5.328633e-19,0.9999327
2593,1,526271,0.4837674,0.8201285,-0.3363611,0.03672778,-9.158219,0.06390385,0.03400733,5.276061e-20,5.159590e-18,0.9998975
2594,1,526449,0.2586836,0.6269798,-0.3682963,0.04512450,-8.161781,0.06071442,0.03270924,3.301202e-16,1.749736e-14,0.9999041
2595,1,526450,0.2586836,0.6269798,-0.3682963,0.04388035,-8.393194,0.05413981,0.04138875,4.731120e-17,2.882265e-15,0.9999373
2596,1,526480,0.2567333,0.6050094,-0.3482762,0.04534077,-7.681302,0.06707697,0.03871426,1.574796e-14,6.303145e-13,0.9994628


## Annotate differential loci

In [15]:
# txdb <- TxDb.Hsapiens.UCSC.hg1overlap="all"9.knownGene
txdb <- EnsDb.Hsapiens.v75
gr_to_annotate <- rowRanges(signif_se) 
# seqlevelsStyle(gr_to_annotate) <- seqlevelsStyle(txdb)
peakAnno <- annotatePeak(
    # rowRanges(signif_se), 
    gr_to_annotate,
    tssRegion = c(-3000, 3000),
    # overlap = "all", # testing see in an attempt to fish for NBEAL2 probes in last exon.
    TxDb = txdb,
    annoDb = "org.Hs.eg.db"
)
peakAnno

>> preparing features information...		 2023-10-15 11:26:21 
>> identifying nearest features...		 2023-10-15 11:26:22 
>> calculating distance from peak to TSS...	 2023-10-15 11:26:32 
>> assigning genomic annotation...		 2023-10-15 11:26:32 
>> adding gene annotation...			 2023-10-15 11:27:32 


'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2023-10-15 11:27:33 
>> done...					 2023-10-15 11:27:33 


Annotated peaks generated by ChIPseeker
1144667/1144667  peaks were annotated
Genomic Annotation Summary:
              Feature   Frequency
9    Promoter (<=1kb) 16.30465454
10   Promoter (1-2kb)  6.20459924
11   Promoter (2-3kb)  3.98683635
4              5' UTR  0.03092602
3              3' UTR  0.72807201
1            1st Exon  0.01092021
7          Other Exon  1.26753021
2          1st Intron 10.47186649
8        Other Intron 19.86053586
6  Downstream (<=300)  0.07888757
5   Distal Intergenic 41.05517150

In [16]:
rowData(signif_se) <- data.frame(peakAnno)
rowData(signif_se)

DataFrame with 1144667 rows and 18 columns
        seqnames     start       end     width   strand        annotation
        <factor> <integer> <integer> <integer> <factor>       <character>
1              1     59275     59275         1        * Distal Intergenic
2              1     85528     85528         1        * Distal Intergenic
3              1    526252    526252         1        *  Promoter (2-3kb)
4              1    526253    526253         1        *  Promoter (2-3kb)
5              1    526270    526270         1        *  Promoter (2-3kb)
...          ...       ...       ...       ...      ...               ...
1144663        Y  58824332  58824332         1        * Distal Intergenic
1144664        Y  58828517  58828517         1        * Distal Intergenic
1144665        Y  58852674  58852674         1        * Distal Intergenic
1144666        Y  58878151  58878151         1        * Distal Intergenic
1144667        Y  58878233  58878233         1        * Distal Interg

In [17]:
# Was useful only when using UCSC annotation
# rowData(signif_se)$seqnames <- sub(
#     "chr",
#     "",
#     rowData(signif_se)$seqnames
# )
# rowData(signif_se)[1:3,]

In [18]:
rowData(signif_se)$simple_annotation <- sub(
    "^(Intron|Exon).*",
    "\\1",
    rowData(signif_se)$annotation
)
rowData(signif_se)$simple_annotation[1:100]

[1] "Distal Intergenic" "Distal Intergenic" "Promoter (2-3kb)" 
  [4] "Promoter (2-3kb)"  "Promoter (2-3kb)"  "Promoter (2-3kb)" 
  [7] "Promoter (2-3kb)"  "Promoter (2-3kb)"  "Promoter (2-3kb)" 
 [10] "Promoter (2-3kb)"  "Intron"            "Intron"           
 [13] "Promoter (2-3kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [16] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [19] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [22] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [25] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [28] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [31] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [34] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [37] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [40] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [43] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [46] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [49] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [52] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [55] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [58] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [61] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [64] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [67] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [70] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [73] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [76] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [79] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [82] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [85] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [88] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [91] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [94] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
 [97] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
[100] "Promoter (<=1kb)"

In [19]:
B <- as.matrix(getCoverage(signif_se, type = "M") / getCoverage(signif_se, type = "Cov"))
B[1:5,1:5]
quantile(B, na.rm = TRUE)

ODG_080,ODG_081,ODG_082,ODG_083,ODG_084
NaN,0.0,NaN,0,0.0000000
0,0.5,1,0,NaN
NaN,1.0,1,NaN,0.6666667
NaN,1.0,1,NaN,NaN
1,1.0,1,NaN,1.0000000


0%       25%       50%       75%      100% 
0.0000000 0.2500000 0.6000000 0.8571429 1.0000000

In [20]:
# M <- as.matrix(getCoverage(signif_se, type = "M"))
# B <- 2^M / (2^M + 1)
# rm(M)
# B[1:5,1:5]

In [99]:
signif_df <- data.frame(rowData(signif_se))

signif_df$coords <- paste(
    signif_df$seqnames,
    signif_df$start,
    sep=":"
)

# auto conv to data.frame duplicate columns...
cols_to_remove <- c(
    "seqnames",
    "start",
    "end",
    "width",
    "strand"
    # 'start_position',
    # 'end_position',
    # 'feature_strand',
    # 'insideFeature',
    # 'distancetoFeature',
)

signif_df[, cols_to_remove] <- NULL

dml_signif_res$coords <- paste(
    dml_signif_res$chr,
    dml_signif_res$pos,
    sep=":"
)

signif_df <- dplyr::left_join(
    signif_df,
    dml_signif_res,
    by = "coords"
)
cols_to_remove <- c(
    'geneChr',
    'geneStart',
    'geneEnd',
    'geneLength',
    'geneStrand',
    # 'symbol',
    # 'chr',
    # 'pos',
    # 'mu1',
    # 'mu2',
    # 'diff',
    'diff.se',
    'stat',
    'phi1',
    'phi2',
    # 'pval',
    # 'fdr',
    'postprob.overThreshold',
    'coords',
    'X'
)

signif_df[, cols_to_remove] <- NULL
signif_df[1:3,]


,annotation,geneId,transcriptId,transcriptBiotype,distanceToTSS,ENTREZID,SYMBOL,GENENAME,simple_annotation,chr,pos,mu1,mu2,diff,pval,fdr
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Distal Intergenic,ENSG00000240361,ENST00000492842,unprocessed_pseudogene,-3673,403263,OR4G11P,olfactory receptor family 4 subfamily G member 11 pseudogene,Distal Intergenic,1,59275,0.4445518,0.7888945,-0.3443427,3.653067e-122,1.962353e-116
2,Distal Intergenic,ENSG00000239945,ENST00000495576,lincRNA,5577,NA,NA,NA,Distal Intergenic,1,85528,0.2659917,0.6063652,-0.3403735,8.635638e-15,3.611938e-13
3,Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2419,NA,NA,NA,Promoter (2-3kb),1,526252,0.5202774,0.8422268,-0.3219494,1.395583e-17,9.272045e-16


In [100]:
annotated_meth_beta <- cbind(
    signif_df,
    round(B,2)
)
colnames(annotated_meth_beta)

[1] "annotation"        "geneId"            "transcriptId"     
 [4] "transcriptBiotype" "distanceToTSS"     "ENTREZID"         
 [7] "SYMBOL"            "GENENAME"          "simple_annotation"
[10] "chr"               "pos"               "mu1"              
[13] "mu2"               "diff"              "pval"             
[16] "fdr"               "ODG_080"           "ODG_081"          
[19] "ODG_082"           "ODG_083"           "ODG_084"          
[22] "ODG_085"           "ODG_086"           "ODG_087"          
[25] "ODG_088"           "ODG_089"           "ODG_090"          
[28] "ODG_091"           "ODG_092"           "ODG_093"          
[31] "ODG_094"           "ODG_095"           "ODG_096"          
[34] "ODG_097"           "ODG_098"           "ODG_099"          
[37] "ODG_100"           "ODG_101"           "ODG_102"          
[40] "ODG_103"           "ODG_104"           "ODG_105"          
[43] "ODG_106"           "ODG_107"           "ODG_108"          
[46] "ODG_109"

In [101]:
annotated_meth_beta$pos[1:3]

[1]  59275  85528 526252

In [102]:
dml_signif_res$pos[1:3]
dml_signif_res[1:3,]

[1]   59275 1935039 1935047

,chr,pos,mu1,mu2,diff,diff.se,stat,phi1,phi2,pval,fdr,postprob.overThreshold,coords
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
910,1,59275,0.4445518,0.7888945,-0.3443427,0.01464996,-23.50468,0.05147487,0.03319496,3.653067e-122,1.962353e-116,1,1:59275
90280,1,1935039,0.6144430,0.1389232,0.4755198,0.03307574,14.37669,0.05504497,0.03334645,7.247457e-47,2.152779e-43,1,1:1935039
90281,1,1935047,0.6137501,0.1361902,0.4775599,0.03291993,14.50671,0.04188551,0.03839120,1.098619e-47,3.585773e-44,1,1:1935047


In [103]:
# Change pos to 1-based coordinates
# Condition there to avoid changing if already 1-based
if (annotated_meth_beta$pos[1] == 59275) {
    annotated_meth_beta$pos <- annotated_meth_beta$pos + 1
}

In [134]:
select(txdb, keys="GRASP", keytype="SYMBOL", columns="GENEID")

SYMBOL,GENEID
<chr>,<chr>
GRASP,ENSG00000161835


In [135]:
xlsx_sheets <- list()

xlsx_sheets$readme <- data.frame(
    Readme = "This file contains the results of the differential methylation analysis for individual CG with pval < 0.001 and |diff| > 0.2. Refer to the 'column_description' sheet for details about the columns. Subsets of the results are provided for the three genes of interest. No CG meets the criteria for the gene 'ANXA3'."
)

col_desc_m <- t(
    data.frame(
        # "column" = "description",
        'start_position' = 'Start position of the associated gene',
        'end_position' = 'Start position of the associated gene',
        'feature_strand' = 'Strand of the associated gene',
        'insideFeature' = 'Is the CG inside the associated gene',
        'distancetoFeature' = 'Distance from CG to the associated gene',
        'symbol' = 'Gene symbol of the associated gene',
        'chr' = 'Chromosome of the CG',
        'pos' = 'Position of the CG',
        'mu1' = 'Mean Beta value for the tumoral group',
        'mu2' = 'Mean Beta value for the normal group',
        'diff' = 'Difference between the mean Beta values',
        'pval' = 'P-value of the differential methylation test',
        'fdr' = 'Adjusted P-value of the differential methylation test',
        'ODG_...' = 'Beta values for each sample, rounded to 2 decimals',
        row.names = NULL
    )
)
xlsx_sheets$columns_description <- data.frame(
    column = rownames(col_desc_m),
    description = col_desc_m,
    row.names = NULL
)

# Too big 200 mo xlsx if added.
xlsx_sheets$all_diff_CG <- annotated_meth_beta

focus_genes <- c(
    "SLC22A3",
    "ANXA3",
    "CUX2",
    "PRICKLE2",
    "TMEM106A",
    "TAMALIN",
    # "GRASP", # working alias for "TAMALIN",
    "NBEAL2",
    "TJP2",
    "SALL3",
    "CLDN5"
)

for (gene in focus_genes) {
    # Need special case because of GRASP alias in use...
    if (gene == "TAMALIN") {
        ensembl_id <- "ENSG00000161835"
    } else {
        # Get the Ensembl gene ID for your Entrez ID
        ensembl_id <- select(txdb, keys=gene, keytype="SYMBOL", columns="GENEID")
        ensembl_id <- ensembl_id$GENEID
    }

    message(ensembl_id)
    message(gene)
    # Note how we need to select either on SYMBOL (If the locus is associated to the TSS of the gene)
    # OR using annotation (if the locus is associated to the gene body)
    xlsx_sheets[[gene]] <- annotated_meth_beta[
        {
            annotated_meth_beta$SYMBOL %in% gene |
            grepl(
                ensembl_id,
                annotated_meth_beta$annotation
            )
        }
        ,
    ]
}

ENSG00000146477

SLC22A3



ENSG00000138772

ANXA3

ENSG00000111249

CUX2

ENSG00000163637

PRICKLE2

ENSG00000184988

TMEM106A

ENSG00000161835

TAMALIN

ENSG00000160796

NBEAL2

ENSG00000119139

TJP2

ENSG00000256463ENSG00000263310

SALL3

Warning message in grepl(ensembl_id, annotated_meth_beta$annotation):
"argument 'pattern' has length > 1 and only the first element will be used"
ENSG00000184113

CLDN5



In [136]:
lapply(
    xlsx_sheets,
    dim
)

$readme
[1] 1 1

$columns_description
[1] 14  2

$all_diff_CG
[1] 1144667      49

$SLC22A3
[1]  4 49

$ANXA3
[1]  3 49

$CUX2
[1] 84 49

$PRICKLE2
[1] 180  49

$TMEM106A
[1] 117  49

$TAMALIN
[1] 237  49

$NBEAL2
[1] 124  49

$TJP2
[1] 264  49

$SALL3
[1] 50 49

$CLDN5
[1] 268  49

In [137]:
# write.xlsx(
#     xlsx_sheets,
#     file = "export_annotated_differential_cg.xlsx"
# )

In [138]:
xlsx_sheets$all_diff_CG <- NULL
write.xlsx(
    xlsx_sheets,
    file = "export_annotated_differential_cg_focus_genes.xlsx"
)

In [139]:
annotated_meth_beta[grepl("4705107", annotated_meth_beta$pos),!grepl("ODG", colnames(annotated_meth_beta))]
annotated_meth_beta[grepl("4705107", annotated_meth_beta$pos),c("annotation","pos")]

,annotation,geneId,transcriptId,transcriptBiotype,distanceToTSS,ENTREZID,SYMBOL,GENENAME,simple_annotation,chr,pos,mu1,mu2,diff,pval,fdr,pmlog10pval,pmlog10fdr,short_annotation
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
187541,Promoter (1-2kb),ENSG00000160796,ENST00000469349,retained_intron,1819,23218,NBEAL2,neurobeachin like 2,Promoter (1-2kb),3,47051073,0.5739843,0.06764876,0.5063355,7.228958e-44,1.526446e-40,30,30,Promoter (1-2kb)
187542,Promoter (1-2kb),ENSG00000160796,ENST00000469349,retained_intron,1820,23218,NBEAL2,neurobeachin like 2,Promoter (1-2kb),3,47051074,0.5737090,0.06770263,0.5060064,1.902218e-43,3.832105e-40,30,30,Promoter (1-2kb)


,annotation,pos
,<chr>,<dbl>
187541,Promoter (1-2kb),47051073
187542,Promoter (1-2kb),47051074


In [140]:
colnames(annotated_meth_beta)

[1] "annotation"        "geneId"            "transcriptId"     
 [4] "transcriptBiotype" "distanceToTSS"     "ENTREZID"         
 [7] "SYMBOL"            "GENENAME"          "simple_annotation"
[10] "chr"               "pos"               "mu1"              
[13] "mu2"               "diff"              "pval"             
[16] "fdr"               "ODG_080"           "ODG_081"          
[19] "ODG_082"           "ODG_083"           "ODG_084"          
[22] "ODG_085"           "ODG_086"           "ODG_087"          
[25] "ODG_088"           "ODG_089"           "ODG_090"          
[28] "ODG_091"           "ODG_092"           "ODG_093"          
[31] "ODG_094"           "ODG_095"           "ODG_096"          
[34] "ODG_097"           "ODG_098"           "ODG_099"          
[37] "ODG_100"           "ODG_101"           "ODG_102"          
[40] "ODG_103"           "ODG_104"           "ODG_105"          
[43] "ODG_106"           "ODG_107"           "ODG_108"          
[46] "ODG_109"           "pmlog10pval"       "pmlog10fdr"       
[49] "short_annotation"

In [141]:
annotated_meth_beta

annotation,geneId,transcriptId,transcriptBiotype,distanceToTSS,ENTREZID,SYMBOL,GENENAME,simple_annotation,chr,⋯,ODG_103,ODG_104,ODG_105,ODG_106,ODG_107,ODG_108,ODG_109,pmlog10pval,pmlog10fdr,short_annotation
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
Distal Intergenic,ENSG00000240361,ENST00000492842,unprocessed_pseudogene,-3673,403263,OR4G11P,olfactory receptor family 4 subfamily G member 11 pseudogene,Distal Intergenic,1,⋯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.000000,30.00000,Distal Intergenic
Distal Intergenic,ENSG00000239945,ENST00000495576,lincRNA,5577,NA,NA,NA,Distal Intergenic,1,⋯,NaN,0.00,1.00,NaN,NaN,NaN,NaN,14.063706,12.44226,Distal Intergenic
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2419,NA,NA,NA,Promoter (2-3kb),1,⋯,0.00,1.00,NaN,0.00,0.00,NaN,NaN,16.855244,15.03282,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2420,NA,NA,NA,Promoter (2-3kb),1,⋯,0.33,NaN,NaN,1.00,NaN,1.00,NaN,15.913835,14.15848,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2437,NA,NA,NA,Promoter (2-3kb),1,⋯,NaN,1.00,NaN,1.00,0.00,NaN,NaN,20.334583,18.27338,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2438,NA,NA,NA,Promoter (2-3kb),1,⋯,0.00,0.00,NaN,NaN,NaN,1.00,NaN,19.277690,17.28738,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2616,NA,NA,NA,Promoter (2-3kb),1,⋯,0.00,0.33,0.00,0.67,0.67,NaN,NaN,15.481328,13.75703,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2617,NA,NA,NA,Promoter (2-3kb),1,⋯,0.00,1.00,0.00,0.20,0.00,NaN,NaN,16.325036,14.54027,Promoter (2-3kb)
Promoter (2-3kb),ENSG00000231709,ENST00000417636,lincRNA,-2647,NA,NA,NA,Promoter (2-3kb),1,⋯,NaN,0.83,0.20,0.00,0.50,0.50,NaN,13.802776,12.20044,Promoter (2-3kb)


In [142]:
xlsx_sheets$NBEAL2[, c("pos")]

[1] 47050427 47050428 47050446 47050447 47050487 47050488 47050491 47050492
  [9] 47050493 47050494 47050522 47050523 47050527 47050528 47050533 47050534
 [17] 47050546 47050547 47050554 47050555 47050587 47050588 47050596 47050597
 [25] 47050598 47050599 47050627 47050628 47050646 47050647 47050650 47050651
 [33] 47050655 47050656 47050660 47050661 47050665 47050666 47050667 47050668
 [41] 47050670 47050671 47050699 47050700 47050712 47050713 47050726 47050727
 [49] 47050728 47050729 47050730 47050731 47050742 47050743 47050749 47050750
 [57] 47050751 47050752 47050754 47050755 47050773 47050774 47050782 47050783
 [65] 47050803 47050804 47050805 47050806 47050823 47050824 47050831 47050832
 [73] 47050838 47050839 47050844 47050845 47050858 47050859 47050868 47050869
 [81] 47050879 47050880 47050882 47050883 47050893 47050894 47050919 47050920
 [89] 47050950 47050951 47050954 47050955 47050978 47050979 47050982 47050983
 [97] 47050991 47050992 47050998 47050999 47051001 47051002 47051013 47051014
[105] 47051034 47051035 47051040 47051041 47051044 47051045 47051073 47051074
[113] 47051083 47051084 47051095 47051096 47051103 47051104 47051113 47051114
[121] 47051120 47051121 47051142 47051143

# Results

In [143]:
epsilon <- 1e-30
annotated_meth_beta$pmlog10pval <- -log10(annotated_meth_beta$pval + epsilon)
annotated_meth_beta$pmlog10fdr <- -log10(annotated_meth_beta$fdr + epsilon)

col_fun_meth <- circlize::colorRamp2(c(0, 0.5, 1), c("blue", "white", "red"))
col_fun_diff <- circlize::colorRamp2(c(-0.5, 0, 0.5), c("blue", "white", "red"))
col_fun_pval <- circlize::colorRamp2(c(0.05, 0), c("white", "#097e09"))
col_fun_pmlog10pval <- circlize::colorRamp2(c(1.3, 30), c("white", "#097e09"))


In [144]:
# This short annotation might be the compromise betwen the "annotation" and "simple_annotation"
annotated_meth_beta$short_annotation <- sub(
    "ENS.*on ",
    "",
    annotated_meth_beta$annotation
)
annotated_meth_beta$short_annotation[1:20]

[1] "Distal Intergenic" "Distal Intergenic" "Promoter (2-3kb)" 
 [4] "Promoter (2-3kb)"  "Promoter (2-3kb)"  "Promoter (2-3kb)" 
 [7] "Promoter (2-3kb)"  "Promoter (2-3kb)"  "Promoter (2-3kb)" 
[10] "Promoter (2-3kb)"  "Intron (1 of 2)"   "Intron (1 of 2)"  
[13] "Promoter (2-3kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
[16] "Promoter (<=1kb)"  "Promoter (<=1kb)"  "Promoter (<=1kb)" 
[19] "Promoter (<=1kb)"  "Promoter (<=1kb)"

In [148]:
for (focus_gene in focus_genes) {
    IRdisplay::display_markdown(paste("#", focus_gene))
    message(focus_gene)

    annotated_meth_beta_subset <- annotated_meth_beta[annotated_meth_beta$SYMBOL %in% focus_gene,]
    if (nrow(annotated_meth_beta_subset) == 0) {
        message("No differential probes associated")
    } else if (nrow(annotated_meth_beta_subset) == 1) {
        message("Only 1 differential probe associated")
    } else {
        set.seed(1)
        ht = Heatmap(
            as.matrix(
                annotated_meth_beta_subset[
                    ,
                    grepl(
                        "ODG_",
                        colnames(annotated_meth_beta_subset)
                    )
                ]
            ),
            col = col_fun_meth,
            cluster_rows = FALSE,
            cluster_columns = FALSE,
            column_split = colData(se)$tissue,
            name = "Methylation\nBeta\nvalues",
            right_annotation = rowAnnotation(
                df = annotated_meth_beta_subset[
                    ,
                    c(
                        "short_annotation",
                        "mu1",
                        "mu2",
                        "diff",
                        "pmlog10fdr"
                    )
                ],
                col = list(
                    "mu1" = col_fun_meth,
                    "mu2" = col_fun_meth,
                    "diff" = col_fun_diff,
                    "pmlog10fdr" = col_fun_pmlog10pval
                )
            ),
            row_labels = paste(
                annotated_meth_beta_subset$chr,
                annotated_meth_beta_subset$pos,
                sep = ":"
            ),
            row_title = paste("Differentially methylated CG associated to", focus_gene),
        )

        set_plot_dim(16, 15)
        # draw(ht)
        filepath <- paste0(focus_gene, "_differential_CG_heatmap.pdf")
        pdf(
            filepath,
            width = 16,
            height = 15
        )
        draw(ht)
        dev.off()
        IRdisplay::display_markdown(
            paste0(
                "Heatmap is available [here](",
                filepath,
                ")."
            )
        )
    }
}

# SLC22A3

SLC22A3

No differential probes associated



# ANXA3

ANXA3



Heatmap is available [here](ANXA3_differential_CG_heatmap.pdf).

...

SALL3



Heatmap is available [here](SALL3_differential_CG_heatmap.pdf).

# CLDN5

CLDN5



Heatmap is available [here](CLDN5_differential_CG_heatmap.pdf).

## Focus on probes selected by Jean-Pierre

In [155]:
# Now we want to focus only on probes selected by JP
JP_probes <- list()
JP_probes$PRICKLE$chr <- 3
JP_probes$PRICKLE$pos <- c(
    64253534,
    64253537,
    64253553,
    64253576,
    64253597,
    64253600,
    64253605,
    64253608,
    64253617,
    64253622
)
JP_probes$TMEM106A$chr <- 17
JP_probes$TMEM106A$pos <- c(
    41363885,
    41363891,
    41363899,
    41363915,
    41363924,
    41363933,
    41363937,
    41363939
)
JP_probes$TAMALIN$chr <- 12
JP_probes$TAMALIN$pos <- c(
    52401199,
    52401207,
    52401214,
    52401223,
    52401234,
    52401236,
    52401246,
    52401262,
    52401264
)
JP_probes$NBEAL2$chr <- 3
JP_probes$NBEAL2$pos <- c(
    47051073,
    47051083,
    47051095,
    47051103,
    47051113,
    47051120,
    47051142,
    47051145,
    47051151
)
JP_probes$TJP2$chr <- 9
JP_probes$TJP2$pos <- c(
    71789293,
    71789298,
    71789307,
    71789309,
    71789322,
    71789325,
    71789346,
    71789348,
    71789367
)
JP_probes$SALL3$chr <- 18
JP_probes$SALL3$pos <- c(
    76738532,
    76738535,
    76738539,
    76738542,
    76738544,
    76738549,
    76738551,
    76738559,
    76738565,
    76738577,
    76738596
)
JP_probes$CLDN5$chr <- 22
JP_probes$CLDN5$pos <- c(
    19512084,
    19512108,
    19512110,
    19512116,
    19512123,
    19512127,
    19512134,
    19512150,
    19512153,
    19512165,
    19512167
)
JP_probes


$PRICKLE
$PRICKLE$chr
[1] 3

$PRICKLE$pos
 [1] 64253534 64253537 64253553 64253576 64253597 64253600 64253605 64253608
 [9] 64253617 64253622


$TMEM106A
$TMEM106A$chr
[1] 17

$TMEM106A$pos
[1] 41363885 41363891 41363899 41363915 41363924 41363933 41363937 41363939


$TAMALIN
$TAMALIN$chr
[1] 12

$TAMALIN$pos
[1] 52401199 52401207 52401214 52401223 52401234 52401236 52401246 52401262
[9] 52401264


$NBEAL2
$NBEAL2$chr
[1] 3

$NBEAL2$pos
[1] 47051073 47051083 47051095 47051103 47051113 47051120 47051142 47051145
[9] 47051151


$TJP2
$TJP2$chr
[1] 9

$TJP2$pos
[1] 71789293 71789298 71789307 71789309 71789322 71789325 71789346 71789348
[9] 71789367


$SALL3
$SALL3$chr
[1] 18

$SALL3$pos
 [1] 76738532 76738535 76738539 76738542 76738544 76738549 76738551 76738559
 [9] 76738565 76738577 76738596


$CLDN5
$CLDN5$chr
[1] 22

$CLDN5$pos
 [1] 19512084 19512108 19512110 19512116 19512123 19512127 19512134 19512150
 [9] 19512153 19512165 19512167

In [158]:
xlsx_sheets <- list()

xlsx_sheets$readme <- data.frame(
    Readme = "This file contains the results of the differential methylation analysis for individual CG selected by Jean-Pierre for further PCR analysis. Refer to the 'column_description' sheet for details about the columns. Subsets of the results are provided for the three genes of interest. No CG meets the criteria for the gene 'ANXA3'."
)

col_desc_m <- t(
    data.frame(
        # "column" = "description",
        'start_position' = 'Start position of the associated gene',
        'end_position' = 'Start position of the associated gene',
        'feature_strand' = 'Strand of the associated gene',
        'insideFeature' = 'Is the CG inside the associated gene',
        'distancetoFeature' = 'Distance from CG to the associated gene',
        'symbol' = 'Gene symbol of the associated gene',
        'chr' = 'Chromosome of the CG',
        'pos' = 'Position of the CG',
        'mu1' = 'Mean Beta value for the tumoral group',
        'mu2' = 'Mean Beta value for the normal group',
        'diff' = 'Difference between the mean Beta values',
        'pval' = 'P-value of the differential methylation test',
        'fdr' = 'Adjusted P-value of the differential methylation test',
        'ODG_...' = 'Beta values for each sample, rounded to 2 decimals',
        row.names = NULL
    )
)
xlsx_sheets$columns_description <- data.frame(
    column = rownames(col_desc_m),
    description = col_desc_m,
    row.names = NULL
)

for (focus_gene in names(JP_probes)) {
    IRdisplay::display_markdown(paste("#", focus_gene))
    message(focus_gene)

    annotated_meth_beta_subset <- annotated_meth_beta[
        {
            annotated_meth_beta$chr %in% JP_probes[[focus_gene]][["chr"]] &
            annotated_meth_beta$pos %in% JP_probes[[focus_gene]][["pos"]]
        },
    ]
    xlsx_sheets[[focus_gene]] <- annotated_meth_beta_subset
    if (nrow(annotated_meth_beta_subset) == 0) {
        message("No differential probes associated")
    } else if (nrow(annotated_meth_beta_subset) == 1) {
        message("Only 1 differential probe associated")
    } else {
        set.seed(1)
        ht = Heatmap(
            as.matrix(
                annotated_meth_beta_subset[
                    ,
                    grepl(
                        "ODG_",
                        colnames(annotated_meth_beta_subset)
                    )
                ]
            ),
            col = col_fun_meth,
            cluster_rows = FALSE,
            cluster_columns = FALSE,
            column_split = colData(se)$tissue,
            name = "Methylation\nBeta\nvalues",
            right_annotation = rowAnnotation(
                df = annotated_meth_beta_subset[
                    ,
                    c(
                        "short_annotation",
                        "mu1",
                        "mu2",
                        "diff",
                        "pmlog10fdr"
                    )
                ],
                col = list(
                    "mu1" = col_fun_meth,
                    "mu2" = col_fun_meth,
                    "diff" = col_fun_diff,
                    "pmlog10fdr" = col_fun_pmlog10pval
                )
            ),
            row_labels = paste(
                annotated_meth_beta_subset$chr,
                annotated_meth_beta_subset$pos,
                sep = ":"
            ),
            row_title = paste("Differentially methylated CG associated to", focus_gene),
        )

        set_plot_dim(16, 7)
        # draw(ht)
        filepath <- paste0(focus_gene, "_JP_differential_CG_heatmap.pdf")
        pdf(
            filepath,
            width = 16,
            height = 7
        )
        draw(ht)
        dev.off()
        IRdisplay::display_markdown(
            paste0(
                "Heatmap is available [here](",
                filepath,
                ")."
            )
        )
    }
}

write.xlsx(
    xlsx_sheets,
    file = "export_annotated_differential_cg_JP_focus_genes.xlsx"
)


# PRICKLE

PRICKLE



Heatmap is available [here](PRICKLE_JP_differential_CG_heatmap.pdf).

# TMEM106A

TMEM106A



...

SALL3



Heatmap is available [here](SALL3_JP_differential_CG_heatmap.pdf).

# CLDN5

CLDN5



Heatmap is available [here](CLDN5_JP_differential_CG_heatmap.pdf).

In [ ]:
sessionInfo()